In [ ]:
%matplotlib inline 

from dolfin import *

class Left(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 0.0)

class Right(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 1.0)

class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], 0.0)

class Top_Left(SubDomain):
    def inside(self, x, on_boundary):
        return (near(x[1], 1.0) and between(x[0], (0, 0.49)))
    
class Top_Center(SubDomain):
    def inside(self, x, on_boundary):
         return (near(x[1], 1.0) and between(x[0], (0.49, 0.51)))
    
class Top_Right(SubDomain):
    def inside(self, x, on_boundary):
         return (near(x[1], 1.0) and between(x[0], (0.51, 1.0)))

# Initialize sub-domain instances
left = Left()
bottom = Bottom()
right = Right()
top_l = Top_Left()
top_c = Top_Center()
top_r = Top_Right()


# Define mesh
mesh = UnitSquareMesh(100, 100)
V = FunctionSpace(mesh, "Lagrange", 1)


# Initialize mesh function for interior domains
domains = MeshFunction("size_t", mesh, mesh.topology().dim())
domains.set_all(0)

# Initialize mesh function for boundary domains
boundaries = MeshFunction("size_t", mesh, mesh.topology().dim()-1)
boundaries.set_all(0)
left.mark(boundaries, 1)
right.mark(boundaries, 3)
bottom.mark(boundaries, 2)
top_l.mark(boundaries, 4)
top_c.mark(boundaries, 5)
top_r.mark(boundaries, 6)


rho_0 = 1450.0
c_0 = 830.0
#convection coefficient. 
h_conv = 11.0
# kappa not in use with steady state solutions 
kappa_0 = 2.03

# Define function space and basis functions
V = FunctionSpace(mesh, "CG", 2)
u = TrialFunction(V)
v = TestFunction(V)

T_0 = 313.0

g_0 = Constant('600000.0')
g_1 = Constant('0')

# Define new measures associated with the interior domains and
# exterior boundaries
dx = Measure('dx', domain=mesh, subdomain_data=domains)
ds = Measure('ds', domain=mesh, subdomain_data=boundaries)


a = kappa_0*dot(grad(u), grad(v))*dx(0) +  kappa_0*h_conv*(u)*v*(ds(1) + ds(2) + ds(3)+ds(4)+ds(5)+ds(6))
L = kappa_0*g_1*v*(ds(6)+ds(4)) + g_0*v*ds(5) + kappa_0*h_conv*(T_0)*v*(ds(1) + ds(2) + ds(3)+ds(4)+ds(5)+ds(6))

#Solve Problem
u = Function(V)
solve(a == L, u)

# Plot solution
import matplotlib.pyplot as plt
plt.figure()
p = plot(u)

plt.colorbar(p)
plt.interactive(True)

plt.show()

